# Sprawozdanie PSD - list 3 #
# <b>Filtr Kalmana</b> #
### Stefan Borek ###
### 10-6-2022 ###


In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as ex
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_squared_error
from IPython import display


## <b> 1.1 Filtr $\alpha $ </b> 

<b>Równanie aktualizacji estymaty stanu:</b>

$\hat x_{N,N} = \hat x_{N,N-1} + \frac{1}{N}(z_{N}-\hat{x}_{N, N-1})$

In [2]:
def alpha_filter(x0, Z, N):
    '''
    * param x0: poczatkowe założenie nt. estymowanego parametru
    * param list `Z`: ciąf obserwacji
    :param int N: liczba obserwacji

    :return x_nn: estymowane wartości w następnych N krokach
    '''
    x_nn = [x0]

    for n in range(1, N):
        x_new = x_nn[n-1] + 1/n * (Z[n] - x_nn[n-1])
        x_nn.append(x_new)
    return x_nn

In [3]:
n = 100

v_m = [20 + np.random.normal(0, .1) for _ in range(n)]
v_real = [20 for _ in range(n)]
x = np.arange(n)
df = pd.DataFrame()
df['num'] = np.arange(n)
df['pomiar prędkości'] = [20 + np.random.normal(0, .5) for _ in range(n)]
df['prędkość rzeczywista'] = [20 for _ in range(n)]
df.head()

,num,pomiar prędkości,prędkość rzeczywista
0,0,20.167016,20
1,1,20.187876,20
2,2,19.668695,20
3,3,20.335405,20
4,4,20.089794,20


In [4]:
X_nn = alpha_filter(v_m[0], v_m, len(v_m))

In [5]:
df['prędkość po filterze alpfa'] = X_nn
df.head()

,num,pomiar prędkości,prędkość rzeczywista,prędkość po filterze alpfa
0,0,20.167016,20,20.063846
1,1,20.187876,20,19.992986
2,2,19.668695,20,20.088057
3,3,20.335405,20,20.108757
4,4,20.089794,20,20.091776


In [6]:
fig1 = ex.scatter(
    df, 
    x='num', 
    y=['pomiar prędkości', 'prędkość po filterze alpfa'], 
    title='Wartość prędkości w kolejnych pomiarach',
    )
fig2 = ex.line(
    df,
    x='num', 
    y='prędkość rzeczywista',
)
fig2.update_traces(line_color='green', name='real')
fig = go.Figure(
    data=fig1.data + fig2.data
    )
fig.update_layout(
    xaxis_title="nr. pomiaru",
    yaxis_title="prędkość [km/h]",
    title='Prędkość w kolejnych pomiarach'
)
fig.show()

## <b>1.2 Filtr $\alpha - \beta $ </b>

<b>Równanie stanu dla położenia </b> \
 $\hat x_{n,n} = \hat{x}_{n,n−1} + \alpha(z_{n} − \hat x_{n,n−1})$

<b>Równanie stanu dla prędkości</b>

  $\hat{\dot x }_{n,n} = \hat{\dot x}_{n,n−1} + \beta(\frac{z_{n} − \hat x_{n,n−1}}{\Delta t})$


Parametr $ \alpha $ i $ \beta $ odpowiadają za precycję radaru, to znaczy są to niepewności pomiarowe.

In [7]:
def alpha_beta_filter(x0, dx0, Z, N, d_t, alpha=0.2, beta=0.1):
    """
    Funkcja aplikująca filtra alfa - beta
    :param x0: początkowa wartość estymowana
    :param dx0: pochodna początkowej wartości
    :param Z: kolejne obserwacje/estymacje
    :param N: liczba obserwacji
    :param float d_t: delat t, czas co który wykonujemy pomiar
    :param alpha: parametr odpowiadający za niepwenosć pomiarową, domyślnie 0.2
    :param beta: parametr odpowiadający za niepwenosć pomiarową, domyślnie 0.1
    :return x_nn: estyator następnego stanu po N krokach
    """
    x_nn = [x0 + d_t * dx0]
    dx_nn = [dx0]
    for n in range(1, N):
        x_new = x_nn[n-1] + alpha * (Z[n] - x_nn[n-1])
        dx_new = dx_nn[n-1] + beta * ( (Z[n]-x_nn[n-1]) / (d_t))

        x_new = x_new + dx_new * d_t

        x_nn.append(x_new)
        dx_nn.append(dx_new)

    return x_nn

### <b> Zadanie parametrów do filtru $ \alpha - \beta $. </b> ###
<b> Starman </b> - to manekin kosmonauty siedzącego w samochodzie Tesla Roadster wystrzelonym na orbitę przez firmę SpaceX🚀. Jego prędkość względem ziemi podawan jest na 7.97 km/s a odległość 322,461,474 km od Ziemi (09-06-2022). Na tej podstawie wygenerujemy dane na których zastosujemy filtr $ \alpha - \beta $.

In [8]:
v = 7.97
delta_time = 5 # co ile wykonujemy krok
s = [322461474] # odległość starmana od ziemi
s_real = [322461474]
t = [1654804370] # w czasie pisania zadania 9-6-2022 ok 22:00 GMT+2
for n in range(1, 100):
    s.append(s[n-1] + delta_time * 7.97 + np.random.normal(0, 1) * 7.97)
    s_real.append(s_real[n-1] + delta_time * 7.97)
    t.append(t[n-1] + delta_time)

In [9]:
X_nn_ab = alpha_beta_filter(
    x0=s[0],
    dx0=v,
    Z=s,
    N=len(s),
    d_t=delta_time,
)

In [10]:
datetime.fromtimestamp(t[0]).strftime('%H:%M:%S %D')

'21:52:50 06/09/22'

In [11]:
df_starman = pd.DataFrame()
df_starman['czas'] = [datetime.fromtimestamp(d).strftime('%H:%M:%S %D') for d in t]
df_starman['odległość estymowana'] = s
df_starman['odległość rzeczywista'] = s_real
df_starman['filtrowana odległość'] = X_nn_ab

df_starman['czas'] = pd.to_datetime(df_starman['czas'])
df_starman.head()

/var/folders/cw/3q5ykvvj1031m_07d0vbll1m0000gr/T/ipykernel_6813/2379898524.py:7: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,czas,odległość estymowana,odległość rzeczywista,filtrowana odległość
0,2022-06-09 21:52:50,3.224615e+08,3.224615e+08,3.224615e+08
1,2022-06-09 21:52:55,3.224615e+08,3.224615e+08,3.224616e+08
2,2022-06-09 21:53:00,3.224616e+08,3.224616e+08,3.224616e+08
3,2022-06-09 21:53:05,3.224616e+08,3.224616e+08,3.224616e+08
4,2022-06-09 21:53:10,3.224616e+08,3.224616e+08,3.224617e+08


In [12]:
fig1 = ex.scatter(
    df_starman, 
    x='czas', 
    y=['odległość estymowana',	'filtrowana odległość'], 
    title='Wartość prędkości w kolejnych pomiarach',
    )
fig2 = ex.line(
    df_starman,
    x='czas', 
    y='odległość rzeczywista',
)

fig2.update_traces(line_color='green', name='real')
fig = go.Figure(
    data=fig1.data + fig2.data
    )
fig.update_layout(
    xaxis_title="czas",
    yaxis_title="odległość [km]",
    title='Odległość <b>Starmana🚀</b> od Ziemi🌍'
)
fig.show()

## <b> 1.3 Filtr $\alpha - \beta - \gamma $ </b>

W przypadku filtru $\alpha - \beta - \gamma $ będziemy uwzględniać stałą wartość przyspieszenia. \
<b> Filtr opisujemy 3 równaniami stanu: </b> 

  $\hat x_{n,n} = \hat{x}_{n,n-1} + \alpha(z_{n} − \hat x_{n,n−1})$


  $\hat{\dot x }_{n,n} = \hat{\dot x}_{n,n−1} + \beta(\frac{z_{n} − \hat x_{n,n−1}}{\Delta t})$


  $\hat{ \ddot {x}}_{n,n}=\hat{ \ddot {x}}_{n,n} + \gamma(\frac{z_n-\hat x_{n,n-1}}{0.5 \Delta t^2})$


Parametr $ \alpha $ i $ \beta $ odpowiadają za precycję radaru, to znaczy są to niepewności pomiarowe.

In [13]:
def alpha_beta_gamma_filter(x0, dx0, Z, N, d_t, alpha=0.5, beta=0.4, gamma=0.1):
    """
    Funkcja aplikująca filtra alfa - beta
    :param x0: początkowa wartość estymowana
    :param dx0: pochodna początkowej wartości
    :param Z: kolejne obserwacje/estymacje
    :param N: liczba obserwacji
    :param float d_t: delat t, czas co który wykonujemy pomiar
    :param alpha: parametr odpowiadający za niepwenosć pomiarową, domyślnie 0.2
    :param beta: parametr odpowiadający za niepwenosć pomiarową, domyślnie 0.1
    :param gamma: parametr gamma 
    :return x_nn: estyator następnego stanu po N krokach
    :return dx_nn: estymator staun dla prędkości w następnych krokach
    """
    x_nn = [x0 + d_t * dx0]
    dx_nn = [dx0]
    ddx_nn = [0]
    for n in range(1, N):
        x_new = x_nn[n-1] + alpha * (Z[n] - x_nn[n-1])
        dx_new = dx_nn[n-1] + beta * ( (Z[n]-x_nn[n-1]) / (d_t))
        ddx_new = ddx_nn[n-1] + gamma * ( (Z[n] - x_nn[n-1]) / (0.5 * d_t ** 2))

        x_nn.append(x_new + dx_new * d_t + ddx_new * (d_t ** 2 / 2))
        dx_nn.append(dx_new + ddx_new * d_t)
        ddx_nn.append(ddx_new)
    
    return x_nn

In [14]:
T = np.arange(0, 300, 5)

v_0 = 0
a = 15 # m/s^2
s_real = [0 + 0.5 * a * t ** 2 for t in T]
s_est = [s + np.random.normal(0, 1) * 0.5 * a * 5 ** 2 for s in s_real]
s_est[0] = abs(s_est[0])
v = a * T

In [15]:
s_abg = alpha_beta_gamma_filter(
    x0=s_est[0],
    dx0=v[0],
    Z=s_est,
    N=len(s_est),
    d_t=delta_time,   
)

s_ab = alpha_beta_filter(
    x0=s_est[0],
    dx0=v[0],
    Z=s_est,
    N=len(s_est),
    d_t=delta_time,   
)

s_a = alpha_filter(
    x0=s_est[0],
    Z=s_est,
    N=len(s_est)
    )

In [16]:
df_falcon = pd.DataFrame()
df_falcon['czas'] = T
df_falcon['odległość rzeczywista'] = s_real
df_falcon['odległość estymowana'] = s_est
df_falcon['filtr alfa'] = s_a
df_falcon['filtr alfa-beta'] = s_ab
df_falcon['filtr alfa-beta-gama'] = s_abg
df_falcon.head()

,czas,odległość rzeczywista,odległość estymowana,filtr alfa,filtr alfa-beta,filtr alfa-beta-gama
0,0,0.0,67.215195,67.215195,67.215195,67.215195
1,5,187.5,159.335474,159.335474,94.851279,159.335474
2,10,750.0,986.808463,573.071968,371.650462,1051.292658
3,15,1687.5,1686.988011,944.377316,864.659473,2349.127354
4,20,3000.0,3403.905292,1559.259310,1856.374720,4694.950035


In [17]:
fig1 = ex.scatter(
    df_falcon, 
    x='czas', 
    y=['filtr alfa', 'filtr alfa-beta', 'filtr alfa-beta-gama'], 
    title='Wartość prędkości w kolejnych pomiarach',
    )
fig2 = ex.line(
    df_falcon,
    x='czas', 
    y='odległość rzeczywista',
)

fig2.update_traces(line_color='green', name='real')
fig = go.Figure(
    data=fig1.data + fig2.data
    )
fig.update_layout(
    xaxis_title="czas[s]",
    yaxis_title="odległość [m]",
    title='Odległość rakiety<b> Falcon 1🚀</b> od Ziemi po starcie'
)
fig.show()

## <b> 2.1 Filtr Kalmana 1-d </b> ##

In [18]:
def kalman_filter_1d(x0, dx0, ddx0, Z, d_t):
    """
    :param float x0: początkowy stan systemu
    :param dx0: początkowa pochodna
    :param ddx0: druga pochodna początkowego stanu
    :param list Z: obserwacje
    :param int d_t: delta t, przyrost czasu w każdym kroku
    """

    p0 = np.var(Z)
    r = np.random.normal(0, p0 ** 2)

    p_nn = [p0]
    kn = p0/(p0 + r)
    x_nn = [x0]
    dx_nn = [dx0]
    ddx_nn = [ddx0]

    for n in range(1, len(Z)):
        # aktualizowanie estymatora stanu
        x_new = x_nn[n-1] + kn * (Z[n] - x_nn[n-1])

        # predykcja
        x_nn.append(x_new + dx_nn[n-1] * d_t + ddx_nn[n-1] * (d_t ** 2 / 2))
        dx_nn.append(dx_nn[n-1] + ddx_nn[n-1] * d_t)
        ddx_nn.append(ddx_nn[n-1])

        # wzmocnienie kalmana
        kn = p_nn[n-1] / (p_nn[n-1] + r)
        
        # aktualizacja kowariancji
        p_nn.append((1 - kn) * p_nn[n-1])

    return x_nn 

In [19]:
df_falcon['kalman 1-d'] = kalman_filter_1d(
    x0=s_est[0],
    dx0=v[0],
    ddx0=a,
    Z=s_est,
    d_t=delta_time
)

In [20]:
fig1 = ex.scatter(
    df_falcon, 
    x='czas', 
    y=['filtr alfa', 'filtr alfa-beta', 'filtr alfa-beta-gama', 'kalman 1-d'], 
    title='Wartość prędkości w kolejnych pomiarach',
    )
fig2 = ex.line(
    df_falcon,
    x='czas', 
    y='odległość rzeczywista',
)

fig2.update_traces(line_color='green', name='real')
fig = go.Figure(
    data=fig1.data + fig2.data
    )
fig.update_layout(
    xaxis_title="czas[s]",
    yaxis_title="odległość [m]",
    title='Odległość rakiety<b> Falcon 1🚀</b> od Ziemi po starcie'
)
fig.show()

In [21]:
columns = df_falcon.columns
columns = list(columns[3:])
columns.remove('filtr alfa')

In [22]:
sqr_errors = []

for f in columns:
    sqr_errors.append(mean_squared_error(df_falcon['odległość rzeczywista'], df_falcon[f]))

errors = pd.DataFrame()
errors['filters'] = columns
errors['error'] = sqr_errors

In [25]:
fig = ex.bar(errors, x='filters', y='error', log_y=True)
fig.show()

In [26]:
errors

,filters,error
0,filtr alfa-beta,9.844506e+07
1,filtr alfa-beta-gama,1.695417e+08
2,kalman 1-d,4.517889e+03
